In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("mps")
print(f"Using {device} device")

Using mps device


In [2]:
class ImageEmbedding(nn.Module):
    def __init__(self, input_channels, embed_dim):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_channels, embed_dim, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

    def forward(self, x):
        return self.conv(x)

In [3]:
class MiniStableDiffusion(nn.Module):
    def __init__(self, embed_dim, artist_dim):
        super().__init__()
        self.embedding = ImageEmbedding(3, embed_dim)
        self.fc1 = nn.Linear(embed_dim + artist_dim, 256)
        self.fc2 = nn.Linear(256, embed_dim)

    def forward(self, x, artist_id):
        x_embed = self.embedding(x)
        print(x_embed.size())

        x_flatten = x_embed.view(x_embed.size(0), -1)
        x_concat = torch.cat((x_flatten, artist_id), dim=1)
        x_fc1 = torch.relu(self.fc1(x_concat))
        x_out = self.fc2(x_fc1)

        return x_out

In [ ]:
# Hyperparameters
embed_dim = 64
artist_dim = 16
learning_rate = 0.001
num_epochs = 10
# Initialize the model, loss function and optimizer
model = MiniStableDiffusion(embed_dim, artist_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# Dummy input image and artist ID
input_image = torch.randn(1, 3, 64, 64).to(device)
artist_id = torch.randn(1, artist_dim).to(device)
# Training loop
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(input_image, artist_id)
    loss = criterion(output, input_image.view(input_image.size(0), -1))
    loss.backward()
    optimizer.step()
    print(f'Epoch: {epoch+1}, Loss: {loss.item()}')
# Generate output image and list of contributing artist IDs
output_image = model(input_image, artist_id)
contributing_artists = [artist_id]
# Print it
print(output_image)
print(contributing_artists)